In [ ]:
pip install gradio networkx matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.5 MB/s eta 0:00:00


In [ ]:
pip install pandas

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import gradio as gr
import pandas as pd

def create_bicyclic_graph(n):
    """Membuat graf bisiklik dengan dua segitiga yang dihubungkan oleh jalur."""
    G = nx.Graph()
    G.add_edges_from([(0, 1), (1, 2), (2, 0)])  # Segitiga pertama
    G.add_edges_from([(3, 4), (4, 5), (5, 3)])  # Segitiga kedua

    if n > 6:
        current_node = 6
        prev_node = 2
        for _ in range(n - 6):
            G.add_edge(prev_node, current_node)
            prev_node = current_node
            current_node += 1
        G.add_edge(prev_node, 3)
    else:
        G.add_edge(2, 3)

    return G

def gutman_index(G):
    """Menghitung indeks Gutman dari graf G."""
    total = 0
    degrees = dict(G.degree())
    nodes = list(G.nodes())
    for i in range(len(nodes)):
        u = nodes[i]
        for j in range(i + 1, len(nodes)):
            v = nodes[j]
            distance = nx.shortest_path_length(G, u, v)
            total += degrees[u] * degrees[v] * distance
    return total

def wiener_index(G):
    """Menghitung indeks Wiener dari graf G."""
    total = 0
    nodes = list(G.nodes())
    for i in range(len(nodes)):
        u = nodes[i]
        for j in range(i + 1, len(nodes)):
            v = nodes[j]
            distance = nx.shortest_path_length(G, u, v)
            total += distance
    return total

def plot_graph(n):
    G = create_bicyclic_graph(n)
    gutman_idx = gutman_index(G)
    wiener_idx = wiener_index(G)

    plt.figure(figsize=(6, 4))
    pos = nx.spring_layout(G)
    nx.draw(G, pos, with_labels=True, node_color='lightblue', edge_color='gray')
    plt.title(f"Bicyclic Graph dengan n={n}\nIndeks Gutman: {gutman_idx}\nIndeks Wiener: {wiener_idx}")
    plt.savefig("graph.png")
    plt.close()

    return "graph.png", f"Indeks Gutman: {gutman_idx}\nIndeks Wiener: {wiener_idx}"

def plot_individual_indices():
    """Memplot grafik masing-masing indeks Gutman dan Wiener terhadap jumlah node."""
    n_values = list(range(6, 21))
    gutman_indices = []
    wiener_indices = []

    for n in n_values:
        G = create_bicyclic_graph(n)
        gutman_indices.append(gutman_index(G))
        wiener_indices.append(wiener_index(G))

    plt.figure(figsize=(6, 4))
    plt.plot(n_values, gutman_indices, marker='o', color='b', linestyle='-', label="Indeks Gutman")
    plt.xlabel("Jumlah Node (n)")
    plt.ylabel("Indeks Gutman")
    plt.title("Grafik Indeks Gutman")
    plt.legend()
    plt.grid()
    plt.savefig("gutman_plot.png")
    plt.close()

    plt.figure(figsize=(6, 4))
    plt.plot(n_values, wiener_indices, marker='o', color='r', linestyle='-', label="Indeks Wiener")
    plt.xlabel("Jumlah Node (n)")
    plt.ylabel("Indeks Wiener")
    plt.title("Grafik Indeks Wiener")
    plt.legend()
    plt.grid()
    plt.savefig("wiener_plot.png")
    plt.close()

    return "gutman_plot.png", "wiener_plot.png"

def plot_combined_indices():
    """Memplot grafik gabungan dari indeks Gutman dan Wiener."""
    n_values = list(range(6, 21))
    gutman_indices = []
    wiener_indices = []

    for n in n_values:
        G = create_bicyclic_graph(n)
        gutman_indices.append(gutman_index(G))
        wiener_indices.append(wiener_index(G))

    plt.figure(figsize=(6, 4))
    plt.plot(n_values, gutman_indices, marker='o', color='b', linestyle='-', label="Indeks Gutman")
    plt.plot(n_values, wiener_indices, marker='s', color='r', linestyle='--', label="Indeks Wiener")
    plt.xlabel("Jumlah Node (n)")
    plt.ylabel("Nilai Indeks")
    plt.title("Grafik Gabungan Indeks Gutman dan Wiener")
    plt.legend()
    plt.grid()
    plt.savefig("combined_plot.png")
    plt.close()

    return "combined_plot.png"

def generate_table():
    """Membuat tabel indeks Gutman dan Wiener untuk berbagai nilai n."""
    n_values = list(range(6, 21))
    gutman_indices = []
    wiener_indices = []

    for n in n_values:
        G = create_bicyclic_graph(n)
        gutman_indices.append(gutman_index(G))
        wiener_indices.append(wiener_index(G))

    df = pd.DataFrame({"Jumlah Node (n)": n_values, "Indeks Gutman": gutman_indices, "Indeks Wiener": wiener_indices})
    return df

def process_input(n):
    graph_img, indices_text = plot_graph(n)
    gutman_plot, wiener_plot = plot_individual_indices()
    combined_plot = plot_combined_indices()
    table_df = generate_table()
    return graph_img, indices_text, gutman_plot, wiener_plot, combined_plot, table_df

iface = gr.Interface(
    fn=process_input,
    inputs=gr.Slider(6, 20, step=1, label="Jumlah Node (n)"),
    outputs=[
        gr.Image(type="filepath", label="Grafik Bicyclic"),
        gr.Textbox(label="Hasil Perhitungan Indeks"),
        gr.Image(type="filepath", label="Grafik Indeks Gutman"),
        gr.Image(type="filepath", label="Grafik Indeks Wiener"),
        gr.Image(type="filepath", label="Grafik Gabungan Indeks"),
        gr.Dataframe(headers=["Jumlah Node (n)", "Indeks Gutman", "Indeks Wiener"], label="Tabel Indeks")
    ],
    title="Bicyclic Graph App",
    description="Masukkan jumlah node (6-20) untuk membentuk graf bisiklik, menghitung indeks, dan melihat grafik perubahan indeks."
)

if __name__ == "__main__":
    iface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d369d56aff3d3f00ae.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
